In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

train_values = pd.read_csv('./Richters_Predictor_Modeling_Earthquake_Damage_-_Train_Values.csv', 
                            index_col='building_id')
train_labels = pd.read_csv('./Richters_Predictor_Modeling_Earthquake_Damage_-_Train_Labels.csv', 
                            index_col='building_id')

In [2]:
from scipy import stats
from sklearn.preprocessing import PowerTransformer
sns.set_style('white')
sns.set_context("paper", font_scale = 2)
#sns.displot(data=train_labels, x="damage_grade", kind="hist", bins = 100, aspect = 1.5)
#for key in train_values.keys():
#    train_values.hist(column=key, bins=100)

pt = PowerTransformer(method='box-cox')
for key in train_values.keys():
    if not np.issubdtype(train_values[key].dtype, np.number):
        continue
    print(key+" values:")
    print(train_values[key].value_counts())
    #fitted_data, fitted_lambda = stats.boxcox(train_values["age"].values+1)
    data = pt.fit_transform(train_values[key].values.reshape(-1, 1)+1)
    #np.histogram(fitted_data)
    #train_values.hist(column="age", bins=100)
    train_values[key] = data
    #plt.hist(data)
    #plt.title(key) 
    #plt.show()

geo_level_1_id values:
6     24381
26    22615
10    22079
17    21813
8     19080
7     18994
20    17216
21    14889
4     14568
27    12532
13     9608
11     8220
3      7540
22     6252
25     5624
16     4332
0      4011
9      3958
12     3194
18     3189
1      2701
5      2690
30     2686
15     2320
14     1714
24     1310
23     1121
2       931
29      396
19      372
28      265
Name: geo_level_1_id, dtype: int64
geo_level_2_id values:
39      4038
158     2520
181     2080
1387    2040
157     1897
        ... 
1330       1
1043       1
637        1
989        1
627        1
Name: geo_level_2_id, Length: 1414, dtype: int64
geo_level_3_id values:
633      651
9133     647
621      530
11246    470
2005     466
        ... 
2366       1
7155       1
10733      1
97         1
3085       1
Name: geo_level_3_id, Length: 11595, dtype: int64
count_floors_pre_eq values:
2    156623
3     55617
1     40441
4      5424
5      2246
6       209
7        39
8         1
9         1
Nam

In [3]:
train_values.dtypes
selected_features = ['foundation_type',
                    'area_percentage',
                    'height_percentage',
                    'count_floors_pre_eq',
                    'land_surface_condition',
                    'has_superstructure_cement_mortar_stone']
train_values_subset = train_values[selected_features]

#sns.pairplot(train_values_subset.join(train_labels), hue='damage_grade')
train_values_subset = pd.get_dummies(train_values_subset)
train_values = pd.get_dummies(train_values)

In [4]:
# for preprocessing the data
from sklearn.preprocessing import StandardScaler

# the model
from sklearn.ensemble import RandomForestClassifier

# for combining the preprocess with model training
from sklearn.pipeline import make_pipeline

# for optimizing the hyperparameters of the pipeline
from sklearn.model_selection import GridSearchCV

In [5]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
import time
from lightgbm import LGBMClassifier
#from xgboost import XGBClassifier
train_values.dtypes
X_train, X_test, y_train, y_test = train_test_split(train_values,train_labels["damage_grade"],test_size=0.3,random_state=1000)

In [6]:
pipe = make_pipeline(StandardScaler(), 
                     RandomForestClassifier(random_state=2018))
param_grid = {'randomforestclassifier__n_estimators': range(200, 500, 100),
              'randomforestclassifier__min_samples_leaf': [1]}
gs = GridSearchCV(pipe, param_grid, cv=10, n_jobs= 8)
gs.fit(X_train, y_train)
gs.best_params_
in_sample_preds = gs.predict(X_test)
f1_score(y_test, in_sample_preds, average='micro')

0.71385630779857